In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm

# Even-odd Hamiltonian - $L=4$

In [2]:
Z = np.array([[1,0],[0,-1]])
Y = np.array([[0,-1.j],[1.j,0]])
X = np.array([[0,1],[1,0]])
I = np.eye(2)

def kron(ops):
    op = 1
    for o in ops:
        op = np.kron(op, o)
    return op

In [114]:
# TFI Hamiltonian
Jz = 1
gx = 1.4
gz = 0.9045
h = Jz*np.kron(Z, Z) + gx/2 * (np.kron(X, I) + np.kron(I, X)) + gz/2 * (np.kron(Z, I) + np.kron(I, Z)) 

In [115]:
h

array([[ 1.9045,  0.7   ,  0.7   ,  0.    ],
       [ 0.7   , -1.    ,  0.    ,  0.7   ],
       [ 0.7   ,  0.    , -1.    ,  0.7   ],
       [ 0.    ,  0.7   ,  0.7   ,  0.0955]])

In [116]:
L, U = np.linalg.eigh(h)
GS = U[:,0]
print(L)
print(GS.conj().T @ h @ GS)

[-1.78630227 -1.          0.53186591  2.25443637]
-1.7863022747631794


In [117]:
psi0 = np.kron(GS, GS)

In [118]:
psi0.conj().T @ (np.kron(h, np.eye(4)) + np.kron(np.eye(4), h)) @ psi0

-3.5726045495263588

In [119]:
L[0]*2

-3.5726045495263588

In [120]:
psi0.reshape(2,2,2,2)

array([[[[ 0.05334253, -0.14062623],
         [-0.14062623,  0.10462136]],

        [[-0.14062623,  0.37073114],
         [ 0.37073114, -0.27581198]]],


       [[[-0.14062623,  0.37073114],
         [ 0.37073114, -0.27581198]],

        [[ 0.10462136, -0.27581198],
         [-0.27581198,  0.20519518]]]])

1. Construct 3 qubit block encoding
2. Apply this to enlarged state of 1.5 * L qubits
3. Determine measurement probability from norm of states
4. 

In [142]:
# Function that does many steps
def HITE(psi, beta, h, max_iter, adaptive=True, force_fail=False, verbose=False, seed=0, periodic=True, correct_iter=3, kill=0):
    # psi is L leg tensor
    # h is two qubit translationally invariant Hamiltonian term
    if seed >= 0:
        np.random.seed(seed)
    d = h.shape[0]
    ell = psi.ndim
    
    L, U = np.linalg.eigh(h) # Energy eigenbasis
    GS = U[:,0]
    Ut = U.reshape(2,2,2,2) # o1, o2, i1, i2
    if ell % 2 == 0:
        # Even number of sites; make pair GS
        GSP = kron([GS] * (ell//2))

    """
    Let a be the minimum of the spectrum. We want 1 - exp(-beta (a + shift)) = kill.
    So when kill = 0, we want shift = -a; then when we fail a measurement, we kill the ground state.
    In general, shift = log(1 - kill) / (-beta) - a
    """
    # Shift spectrum of H to be non-negative with GS energy being 0
    h_diag = np.diag(L)
    shift = np.log(1 - kill) / (-beta) - np.min(L)
    h_diag += shift * np.eye(d)
    #h_diag -= np.min(L) * np.eye(d)
    if verbose:
        print(f"Shifted Spectrum with shift {shift}: {np.diag(h_diag)}.")
    assert np.isclose((U.conj().T @ GS).conj().T @ h_diag @ (U.conj().T @ GS), np.min(L) + shift), GS.conj().T @ h_diag @ GS

    def BE(beta):
        # Make gamma = e^{-beta H_diag)
        gamma = sp.linalg.expm(-beta * h_diag)
        gammat = gamma.reshape(2,2,2,2)
        gamma_bar = np.sqrt(np.eye(d) - gamma**2)
        gamma_bart = gamma_bar.reshape(2,2,2,2)
        U_gamma = np.zeros((2*d,2*d), dtype=gamma.dtype)
        U_gamma[:d,:d] = gamma
        U_gamma[d:,:d] = U_gamma[:d,d:] = gamma_bar
        U_gamma[d:,d:] = -gamma
        assert np.isclose(np.linalg.norm(U_gamma @ U_gamma.conj().T - np.eye(2*d)), 0.0)
        return gammat, gamma_bart

    assert periodic==True
    h_terms = [h] + [I]*(ell-2)
    h_big = kron(h_terms)
    h_big = h_big.reshape([2]*(2*ell))
    H = 0
    for i in range(ell - (not periodic)):
        H += h_big
        h_big = h_big.transpose(list(range(1, ell)) + [0] + list(range(ell+1, 2*ell)) + [ell])
        
    H = H.reshape(2**ell, -1)

    LW, UW = np.linalg.eigh(H) # Energy eigenbasis
    if verbose:
        print(f"GS Energy: {LW[0]}.")
    GSW = UW[:,0]

    psiF = psi.flatten()
    Es = [[[psiF.conj().T @ H @ psiF]]] # Initial energy
    Fs = [[[np.abs(GSW.conj().T @ psiF)**2]]]
    Fs_P = [[[np.abs(GSP.conj().T @ psiF)**2]]]
    
    if verbose:
        print(f"Energy at iteration {0}: {Es[-1]}.")
        print(f"Fidelity at iteration {0}: {Fs[-1]}.")
        print(f"Pair fidelity at iteration {0}: {Fs_P[-1]}.")
        
    mos = []
    def permutation(fail_count, d):
        # Define one particular choice of permutation matrix
        P = np.eye(d)
        if fail_count == 0:
            P = np.roll(np.eye(d), 1, axis=1)
        else:
            P[:-fail_count,:-fail_count] = np.roll(P[:-fail_count,:-fail_count], 1, axis=1)
        return P
    for j in (tqdm(range(1, max_iter+1)) if verbose else range(1, max_iter+1)):
        E_layer = []
        F_layer = []
        FP_layer = []
        mo_layer = []
        for oe in range(3):
            # even odd Trotterization; second order
            if oe == 0 or oe == 2:
                gammat, gamma_bart = BE(beta/2)
            elif oe == 1:
                gammat, gamma_bart = BE(beta)
            else:
                assert False
            
            for i in range(0, ell, 2):
                E_gate = []
                F_gate = []
                FP_gate = []
                mo_gate = []
                fail_count = 0
                #for k in range(4):
                k = 0
                ci_local = correct_iter
                while k < ci_local:
                    # apply gate to (i, i+1)
        
                    # First move to local 2-site energy eigenbasis.
                    psi = np.tensordot(Ut.conj(), psi, axes=([0,1],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                    reorder = list(range(2, i+2)) + [0,1] + list(range(i+2, ell))
                    psi = psi.transpose(reorder)
        
                    gamma_psi = np.tensordot(gammat, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                    gamma_psi = gamma_psi.transpose(reorder)
                    gpF = gamma_psi.flatten()
                    p0 = np.linalg.norm(gpF)**2
                    p1 = 1 - p0
                    if p1 < 1.e-14:
                        p1 = 0
                        p0 = 1
                    mo = np.random.choice([0,1], size=1, p=[p0,p1])[0]
                    if mo == 1 and kill == 0:
                        # Want to see 3 successful measurements before going on.
                        k = 0
                        ci_local *= 2
                    if p1 > 1.e-10 and force_fail:
                        mo = 1
                    mo_gate.append(mo)
                    if verbose:
                        print(f"Measurement outcome and probability at round {k}: {mo}, {[p0,p1][mo]}.")
        
                    if mo == 0:
                        psi = gamma_psi
                    else:
                        gamma_psi = np.tensordot(gamma_bart, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                        gamma_psi = gamma_psi.transpose(reorder)
                        psi = gamma_psi
                    assert np.isclose(np.linalg.norm(psi)**2, [p0,p1][mo])
                    psi /= np.linalg.norm(psi)
        
                    if mo == 1 and adaptive and kill == 0:
                        # Made measurement error; try to fix
                        nP = permutation(fail_count, d)
                        assert np.isclose(np.linalg.norm(nP @ nP.conj().T - np.eye(d)), 0.0)
                        nP = nP.reshape(2,2,2,2)
                        psi = np.tensordot(nP, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                        psi = psi.transpose(reorder)
                        fail_count += 1
                    
                    # Move back to compoutational basis on 2-qubit subspace
                    psi = np.tensordot(Ut, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                    psi = psi.transpose(reorder)
        
                    psiF = psi.flatten()
                    E_gate.append(psiF.conj().T @ H @ psiF)
                    F_gate.append(np.abs(GSW.conj().T @ psiF)**2)
                    FP_gate.append(np.abs(GSP.conj().T @ psiF)**2)
                    
                    if len(E_gate) > 1 and np.abs(E_gate[-1] - E_gate[-2]) < 1.e-8:
                        break
                    k += 1
                if verbose:
                    print(f"Energy at layer {j}, sublayer {oe}, gate {i+oe%2}: {E_gate[-1]}.")
                    print(f"Fidelity at layer {j}, sublayer {oe}, gate {i+oe%2}: {F_gate[-1]}.")
                    print(f"Pair fidelity at layer {j}, sublayer {oe}, gate {i+oe%2}: {FP_gate[-1]}.")
                E_layer.append(E_gate)
                F_layer.append(F_gate)
                FP_layer.append(FP_gate)
                mo_layer.append(mo_gate)
                if len(E_layer) > 1 and np.abs(np.mean(E_layer[-1]) - np.mean(E_layer[-2])) < 1.e-8:
                    break
                if oe == 0:
                    psi = psi.transpose(np.roll(list(range(ell)), 1))
                elif oe == 1:
                    psi = psi.transpose(np.roll(list(range(ell)), -1))
                
        Es.append(E_layer)
        Fs.append(F_layer)
        Fs_P.append(FP_layer)
        mos.append(mo_layer)
        if len(Es) > 2 and np.abs(np.mean(Es[-1][-1]) - np.mean(Es[-2][-1])) < 1.e-8:
            break
        
    return Es, Fs, Fs_P, mos, psi

In [143]:
L = 4
psi0 = np.random.rand(2**L) + 1.j * np.random.rand(2**L)
psi0 /= np.linalg.norm(psi0)

In [144]:
Es_C, Fs_C, Fs_P_C, mos_C, psi_C = HITE(psi0.reshape([2]*L), 5, h, 1, adaptive=True, force_fail=False, verbose=True, seed=-1, correct_iter=3, kill=0.)

Shifted Spectrum with shift 1.7863022747631794: [0.         0.78630227 2.31816818 4.04073864].
GS Energy: -6.742585317119429.
Energy at iteration 0: [[(4.197111526337993+4.440892098500626e-16j)]].
Fidelity at iteration 0: [[0.01435496900674648]].
Pair fidelity at iteration 0: [[0.007794493918193305]].


100%|██████████| 1/1 [00:00<00:00, 47.63it/s]

Measurement outcome and probability at round 0: 1, 0.9492519147165681.
Measurement outcome and probability at round 0: 1, 0.9126397013932638.
Measurement outcome and probability at round 0: 1, 0.6159168179020654.
Measurement outcome and probability at round 1: 0, 1.
Energy at layer 1, sublayer 0, gate 0: (-0.5580928626471036+0j).
Fidelity at layer 1, sublayer 0, gate 0: 0.019794526674989023.
Pair fidelity at layer 1, sublayer 0, gate 0: 0.0022868958366505368.
Measurement outcome and probability at round 0: 0, 0.12829622719361353.
Measurement outcome and probability at round 1: 0, 0.9393594518671917.
Measurement outcome and probability at round 2: 0, 0.9987343645963869.
Energy at layer 1, sublayer 0, gate 2: (-5.117919881964348+4.440892098500626e-16j).
Fidelity at layer 1, sublayer 0, gate 2: 0.5750080803224465.
Pair fidelity at layer 1, sublayer 0, gate 2: 0.5540305648214561.
Measurement outcome and probability at round 0: 1, 0.4071208546521773.
Measurement outcome and probability at r

In [145]:
Fs_C[0]

[[0.01435496900674648]]

In [146]:
[Fs_C[i][-1][-1] for i in range(2)]

[0.01435496900674648, 0.9060430494946661]

In [149]:
Es_F, Fs_F, Fs_P_F, mos_F, psi_F = HITE(psi_C.reshape([2]*L), 0.05, h, 10, adaptive=True, force_fail=False, verbose=True, seed=-1, correct_iter=3, kill=0.)

Shifted Spectrum with shift 1.7863022747631794: [0.         0.78630227 2.31816818 4.04073864].
GS Energy: -6.742585317119429.
Energy at iteration 0: [[(-6.133244503038831+8.881784197001252e-16j)]].
Fidelity at iteration 0: [[0.9060430494946661]].
Pair fidelity at iteration 0: [[0.9999996052942336]].


100%|██████████| 10/10 [00:00<00:00, 107.58it/s]

Measurement outcome and probability at round 0: 0, 0.9999999854949979.
Measurement outcome and probability at round 1: 0, 0.9999999860541986.
Measurement outcome and probability at round 2: 0, 0.9999999865918414.
Energy at layer 1, sublayer 0, gate 0: (-6.133244368773413+0j).
Fidelity at layer 1, sublayer 0, gate 0: 0.9060430528262421.
Pair fidelity at layer 1, sublayer 0, gate 0: 0.9999996471531802.
Measurement outcome and probability at round 0: 0, 0.9765598157258317.
Measurement outcome and probability at round 1: 0, 0.9794018909538894.
Measurement outcome and probability at round 2: 0, 0.9818900498957959.
Energy at layer 1, sublayer 0, gate 2: (-6.35127695306527+4.440892098500626e-16j).
Fidelity at layer 1, sublayer 0, gate 2: 0.938470297746212.
Pair fidelity at layer 1, sublayer 0, gate 2: 0.7845628684485121.
Measurement outcome and probability at round 0: 0, 0.9992888107167989.
Measurement outcome and probability at round 1: 0, 0.9994268924823441.
Measurement outcome and probabil

In [151]:
[Fs_F[i][-1][-1] for i in range(11)]

[0.9060430494946661,
 0.961953451261816,
 0.974035943972905,
 0.978531524479826,
 0.980805201051268,
 0.9820984675870416,
 0.9828712232695088,
 0.9833448968974431,
 0.9836396824742009,
 0.9838249213734362,
 0.983942071248481]

# Average Fidelity

In [130]:
realizations = 100
Fs = []
for i in tqdm(range(realizations)):
    L = 4
    psi0 = np.random.rand(2**L) + 1.j * np.random.rand(2**L)
    psi0 /= np.linalg.norm(psi0)

    Es_C, Fs_C, Fs_P_C, mos_C, psi_C = HITE(psi0.reshape([2]*L), 10, h, 1, adaptive=True, force_fail=False, verbose=False, seed=-1, correct_iter=2)
    Es_F, Fs_F, Fs_P_F, mos_F, psi_F = HITE(psi_C.reshape([2]*L), 0.1, h, 5, adaptive=True, force_fail=False, verbose=False, seed=-1, correct_iter=1, kill=0.4)
    Es_F2, Fs_F2, Fs_P_F2, mos_F2, psi_F2 = HITE(psi_F.reshape([2]*L), 0.01, h, 50, adaptive=True, force_fail=False, verbose=False, seed=-1, correct_iter=1, kill=0.4)
    Fs.append([Fs_C[i][-1][-1] for i in range(2)] + [Fs_F[i][-1][-1] for i in range(1, 6)] + [Fs_F2[i][-1][-1] for i in range(1, 51)])

100%|██████████| 100/100 [00:11<00:00,  8.97it/s]


In [131]:
np.array(Fs).mean(axis=0)

array([0.01330486, 0.90604279, 0.90207619, 0.89521182, 0.89576933,
       0.88441479, 0.88684459, 0.88613356, 0.88582088, 0.8871896 ,
       0.88677632, 0.88587966, 0.88624032, 0.88552731, 0.88525426,
       0.88571245, 0.8858111 , 0.88685333, 0.88772928, 0.8876164 ,
       0.88743543, 0.88689327, 0.88773876, 0.88713614, 0.88682254,
       0.88690969, 0.8883725 , 0.88800676, 0.88926188, 0.88986699,
       0.88968146, 0.88721015, 0.88766213, 0.88638914, 0.88630733,
       0.88674954, 0.88729232, 0.8868392 , 0.88744014, 0.88767263,
       0.8886242 , 0.88860583, 0.88918754, 0.88875717, 0.8888242 ,
       0.88743435, 0.88759152, 0.88845233, 0.88890393, 0.8899622 ,
       0.88913122, 0.88878668, 0.88929855, 0.88840191, 0.88808432,
       0.88725675, 0.88721251])

In [73]:
np.array(Fs)[:,-1]

array([4.58115479e-01, 7.09574120e-01, 1.51358200e-02, 8.04257258e-01,
       1.53343854e-04, 3.37089032e-03, 4.65984892e-01, 3.22595691e-01,
       8.95521225e-03, 5.77136166e-01, 5.73182889e-01, 2.83022744e-01,
       1.94986603e-03, 4.68099709e-03, 8.28201074e-03, 1.71640201e-01,
       8.90512043e-01, 1.36996026e-02, 6.04130586e-01, 3.01124658e-02,
       8.95680480e-01, 1.80040970e-02, 3.66661680e-01, 6.10224647e-01,
       8.27305146e-01, 2.89903046e-02, 1.90547008e-02, 9.87336468e-01,
       7.10821962e-03, 8.87474662e-01, 5.48818573e-02, 7.61686289e-01,
       3.62661490e-01, 5.85311769e-01, 7.73800582e-02, 9.66632481e-01,
       5.58600530e-01, 5.49799400e-01, 1.66801159e-01, 3.16962431e-01,
       2.18620612e-01, 8.28787533e-01, 6.52112630e-01, 9.86023033e-01,
       7.13921128e-01, 1.51949363e-01, 6.46372168e-01, 8.02990889e-01,
       1.02793627e-02, 1.24256768e-01, 3.58758349e-01, 9.05425873e-01,
       1.28948325e-01, 8.26445741e-01, 4.87793479e-01, 7.55590162e-01,
      